In [1]:
# Import API modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
import geopandas as gpd
# Import linestrings
import tradeAreasPreprocessing as tap
import tradeAreasPreprocessing_soheil as tap_s
from shapely.geometry import LineString
import census_finder

In [2]:
sf = gpd.read_file('San_Francisco_Bay_Region_2020_Census_Tracts.geojson')
# rename sf['geoid'] to 'GEOID'
sf.rename(columns={'geoid': 'GEOID'}, inplace=True)

# Get the token
with open('inrix_token.txt', 'r') as f:
    token = f.read()

# read in files
geojson_location = "/Users/ekinokos2/Library/CloudStorage/OneDrive-UW/GeoAdVisor/sf-tracts-2020-clipped.geojson"
census_data = "/Users/ekinokos2/Library/CloudStorage/OneDrive-UW/GeoAdVisor/census_data/census_data_clean.csv"

# Load the GeoJSON file into a GeoDataFrame and csv into df
gdf = gpd.read_file(geojson_location)
census_df = pd.read_csv(census_data, dtype={"tract": str})


In [3]:
# Get each x and y coordinate from geometry column
gdf['x'] = gdf['geometry'].apply(lambda x: x.centroid.x)
gdf['y'] = gdf['geometry'].apply(lambda x: x.centroid.y)

cf = census_finder.censusFinder(gdf, census_df)

def append_demographics(row):
    tract_no = cf.get_census_tract_info(row['start_lat'], row['start_lon'])
    demographics = cf.lookup_demographics(tract_no)
    return demographics.iloc[0]

common_df = pd.DataFrame()

for i in range(len(gdf)):
    try:
        # Get x and y coordinates
        x = gdf['x'][i]
        y = gdf['y'][i]
        tap_df = tap.tradeAreasPreprocessing(token=token,
                                    od = 'origin', 
                                    geoFilterType = 'circle',
                                    radius = '0.9km',
                                    points = f'{y}%7C{x}',
                                    providerType = 'consumer',
                                    startDateTime = '%3E%3D2023-06-01T02%3A31',
                                    endDateTime = '%3C%3D2023-06-15T02%3A31')
        df = tap_df.read_data()
        df = tap_df.create_dataframe(df)
        df = tap_df.separate_coordinates(df)
        df = tap_df.add_temporal_vars(df)
        trips_gdf = tap_df.get_trip_geometry(df)
        #trips_gdf = tap_df.get_multiline_trip_geometry(df)
        intersected_trips = tap_df.intersectedTrips(trips_gdf, sf)
        intersected_trips = pd.merge(intersected_trips, df, on='tripId', how='left')
        demographics_df = df.apply(append_demographics, axis=1)
        result_df = pd.concat([intersected_trips, demographics_df], axis=1)
        # Append to the common DataFrame
        common_df = pd.concat([common_df, result_df], ignore_index=True)
    except:
       print(f'Error at {i}')
       continue
    
    

Error at 3
Error at 5
Error at 6
Error at 7
Error at 8
Error at 21
Error at 32
Error at 93
Error at 94
Error at 95
Error at 96
Error at 100
Error at 108
Error at 109
Error at 110
Error at 111
Error at 124
Error at 125
Error at 127
Error at 128
Error at 129
Error at 130
Error at 132
Error at 135
Error at 137
Error at 149
Error at 150
Error at 151
Error at 152
Error at 153
Error at 154
Error at 171
Error at 183
Error at 198
Error at 199
Error at 200
Error at 231
Error at 232
Error at 236
Error at 240


In [4]:
common_df

,tripId,GEOID,objectid,statefp,countyfp,trctce,name_,namelsad,mtfcc,funcstat,...,Median Household Income,total_pop,pct_female,pct_male,pct_children,pct_20-39,pct_39-64,pct_65+,pct_nonwhite,pct_latine
0,0017faab3a4aa382d42c500d4df4d145,06075023300,607,06,075,None,233,Census Tract 233,G5020,S,...,133458.0,4195.0,0.525387,0.474613,0.080810,0.019547,0.206436,0.103218,0.891061,0.174970
1,018b709dea927be92356560ee9569c53,06075023300,607,06,075,None,233,Census Tract 233,G5020,S,...,133458.0,4195.0,0.525387,0.474613,0.080810,0.019547,0.206436,0.103218,0.891061,0.174970
2,018b709dea927be92356560ee9569c53,06001990000,925,06,001,None,9900,Census Tract 9900,G5020,S,...,67279.0,3930.0,0.502036,0.497964,0.114758,0.025700,0.166667,0.093130,0.889313,0.215522
3,018b709dea927be92356560ee9569c53,06075023400,1044,06,075,None,234,Census Tract 234,G5020,S,...,89125.0,4825.0,0.472539,0.527461,0.114197,0.039585,0.195855,0.116269,0.861554,0.081036
4,018b709dea927be92356560ee9569c53,06001441503,426,06,001,None,4415.03,Census Tract 4415.03,G5020,S,...,67279.0,3930.0,0.502036,0.497964,0.114758,0.025700,0.166667,0.093130,0.889313,0.215522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019049,9572916324f8d58444be8478479dad32,06075013500,118,06,075,None,135,Census Tract 135,G5020,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019050,e2b02a837e622c66ec9c481d252aaffe,06075013500,118,06,075,None,135,Census Tract 135,G5020,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019051,22213d2051b1ee94eb9bd69d0cc644cc,06075013002,816,06,075,None,130.02,Census Tract 130.02,G5020,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019052,3c755797a1e2ff9a013a05e0cab3cfd7,06075013002,816,06,075,None,130.02,Census Tract 130.02,G5020,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
common_df

tripId                         0
GEOID                          0
objectid                       0
statefp                        0
countyfp                       0
trctce                     18626
name_                          0
namelsad                       0
mtfcc                          0
funcstat                       0
aland                          0
awater                         0
intptlat                       0
intptlon                       0
SHAPE_Length                   0
SHAPE_Area                     0
geometry_x                     0
geometry_y                     0
line_locs                      1
providerHash                   0
startLoc                       0
startDateTime                  0
endLoc                         0
endpointType                   0
tripDistanceMeters             0
tripMeanSpeedKPH               0
endDateTime                    0
deviceId                       0
providerType                   0
start_lat                      0
start_lon 

In [71]:
common_df = pd.read_csv('origin_demographics.csv')

In [73]:
common_df['hour'] = pd.to_datetime(common_df['startDateTime']).dt.hour
common_df['day'] = pd.to_datetime(common_df['startDateTime']).dt.day

In [96]:
means = common_df[['hour', 'day', 'tract', 'Median Household Income', 'total_pop', 'pct_female',
       'pct_male', 'pct_children', 'pct_20-39', 'pct_39-64', 'pct_65+',
       'pct_nonwhite', 'pct_latine']].groupby(by=['hour',  'tract']).mean()

In [97]:
counts = common_df[['hour', 'day', 'tract', 'Median Household Income', 'total_pop', 'pct_female',
       'pct_male', 'pct_children', 'pct_20-39', 'pct_39-64', 'pct_65+',
       'pct_nonwhite', 'pct_latine']].groupby(by=['hour',  'tract']).count()

In [99]:
# Multiply counts and means
totals = means * counts

In [100]:
totals.reset_index(inplace=True)

In [111]:
totals.to_csv('totals_for_kaitlyn.csv')

In [29]:
common_df.to_csv('intersected_demographics.csv', index=False)

In [2]:
HASH_TOKEN = 'eGo4dWlyaWk1anxFSkVYZmpoZ0ZZNUNxTTYxNTBqOTU2MlNldmJVRWpxMjVlUEIxd042'
APP_ID = 'xj8uirii5j'
TOKEN_URL = 'https://api.iq.inrix.com/auth/v1/appToken'
def get_token():
    #Pass in the app_id and hash_token as query parameters
    params = {'appId': APP_ID,
              'hashToken': HASH_TOKEN}
    response = requests.get(TOKEN_URL, params=params)
    response.raise_for_status()  # Raise HTTPError for bad responses
    data = response.json()
    token = data['result']['token']
    return token, response.status_code

In [ ]:
df = tap_df.create_dataframe(df)

In [ ]:
df = tap.separate_coordinates(df)

df = tap.add_temporal_vars(df)

#df = tap.add_mode(df)

In [ ]:
trips_gdf = tap.get_trip_geometry(df)

ca_tiger = tap.downloadShapefile()

intersected_trips = tap.intersectedTrips(trips_gdf, ca_tiger)

In [ ]:
# read in files
geojson_location = "/Users/ekinokos2/Library/CloudStorage/OneDrive-UW/GeoAdVisor/sf-tracts-2020-clipped.geojson"
census_data = "/Users/ekinokos2/Library/CloudStorage/OneDrive-UW/GeoAdVisor/census_data/census_data_clean.csv"

# Load the GeoJSON file into a GeoDataFrame and csv into df
gdf = gpd.read_file(geojson_location)
census_df = pd.read_csv(census_data, dtype={"tract": str})

# Filter out invalid geometries
gdf = gdf[gdf.is_valid]

# INitialize class
cf = census_finder.censusFinder(gdf, census_df)

tract_no = cf.get_census_tract_info(37.734622, -122.471603)

demographics = cf.lookup_demographics(tract_no)

In [ ]:
def append_demographics(row):
    tract_no = cf.get_census_tract_info(row['start_lat'], row['start_lon'])
    demographics = cf.lookup_demographics(tract_no)
    return demographics.iloc[0]

demographics_df = df.apply(append_demographics, axis=1)

result_df = pd.concat([df, demographics_df], axis=1).iloc[:, :37]

In [ ]:
# Merge result_df and intersected trips on tripId
intersected_trips_demo = pd.merge(result_df, intersected_trips, on='tripId')

In [ ]:
intersected_trips_demo

## BIG FOR LOOP

KeyError: 'result'

In [ ]:
x

In [ ]:
y

In [ ]:

# Initialize class
tap_df = tap.tradeAreasPreprocessing(token=token,
                                  od = 'origin', 
                                  geoFilterType = 'circle',
                                radius = '0.2km',
                                points = '37.734622%7C-122.471603',
                                providerType = 'consumer',
                                startDateTime = '%3E%3D2023-06-01T02%3A31',
                                endDateTime = '%3C%3D2023-06-15T02%3A31')

# Read data
df = tap_df.read_data()

In [ ]:
df

In [ ]:
f'{y:.6f}%7C{x:.6f}'

In [ ]:
# Get the first six decimals of x
df

In [ ]:
"{:.6f}".format(x)